In [8]:
import importlib
import data_utils
importlib.reload(data_utils)

<module 'data_utils' from '/mnt/projects/2025_16_01_midi_exps/data_utils.py'>

In [12]:
import matplotlib.pyplot as plt

from dataset import get_data_loaders_for_training
from data_utils import HDF_FILE_PATH

batch_size = 1
num_workers = 1

_, _, dev_loader = get_data_loaders_for_training(HDF_FILE_PATH, batch_size, num_workers)

In [23]:
from data_utils import visualize_piano_roll
from models.transformer import TransformerDecoderModel, MODEL_CHECKPOINTS_PATH
from models.configs import transformer_v1_config
from torch import nn
import torch

N_CONTEXT_FRAMES = 200
N_PREDICTION_STEPS = 300
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TransformerDecoderModel(**transformer_v1_config["model_opts"])
model.load_state_dict(torch.load(f"{MODEL_CHECKPOINTS_PATH}/transformer_v1_boosted-mse.pt"))
model.to(DEVICE)
criterion = nn.MSELoss()

def piano_rolls_tensor_to_numpy(piano_rolls_tensor: torch.Tensor):
  return piano_rolls_tensor.permute(0, 2, 1)[0].numpy()


for batch_idx, piano_rolls in enumerate(dev_loader):
    visualize_piano_roll(
        piano_rolls_tensor_to_numpy(piano_rolls),
    )

    with torch.no_grad():
        # context frames to start prediction
        piano_rolls_context = piano_rolls[:, :N_CONTEXT_FRAMES].to(DEVICE)
        # create tensor to hold the predicted tensor
        predicted_piano_rolls = torch.zeros((piano_rolls.size(0), N_CONTEXT_FRAMES + N_PREDICTION_STEPS + 1, piano_rolls.size(2)), device=DEVICE)
        # insert context into tensor while considering initial 0 padding frame
        predicted_piano_rolls[:, 1:N_CONTEXT_FRAMES+1] = piano_rolls_context

        for step in range(N_PREDICTION_STEPS):
            outputs = model(predicted_piano_rolls[:, :N_CONTEXT_FRAMES + step + 1])
            predicted_piano_rolls[:, N_CONTEXT_FRAMES + step + 1] = outputs[:, -1]

        predicted_piano_rolls = predicted_piano_rolls[:, :-1].to("cpu")

    visualize_piano_roll(
        piano_rolls_tensor_to_numpy(predicted_piano_rolls),
    )

    loss = criterion(predicted_piano_rolls, piano_rolls)  # Example target as input
    print("Loss: ", loss)
    
    error = predicted_piano_rolls - piano_rolls
    error[piano_rolls != 0] *= 2
    loss_boosted = torch.mean(error ** 2)
    print("loss_boosted: ", loss_boosted)

    break

FileNotFoundError: [Errno 2] No such file or directory: 'model_checkpoints/transformer_v1_boosted-mse.pt'